<a href="https://colab.research.google.com/github/Heiley-W/Gravitational-Wave-Open-Data-Workshop/blob/main/summary/GW2_Q-transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.